In [1]:
from tqdm import tqdm, trange
import numpy as np

In [11]:
def process_file(input_file, output_file):
    """
    The function process the data files for Delete & Generate and convert
    it for the Delete, Retrieve and Generate training by separating the content
    and attributes. It includes all the attribure words.
    
    Input_file: string : Path of the input file
    Output_file: string : Path of the output file 
    """
    
    with open(input_file) as fp:
        data = fp.read().splitlines()
    with open (output_file,"w") as out_fp:
        for x in tqdm(data):
            temp = x.split("<START>")
            con = temp[0].replace("<POS>","").replace("<NEG>","").replace("<CON_START>","")
            sen = temp[1].replace("<END>","")
            lt1 = con.split()
            lt2 = sen.split()
            att_tokens = [z for z in lt2 if z not in lt1]
            max_atts = 0
            if len(att_tokens) > max_atts:
                max_atts = len(att_tokens)
            att_words = " ".join(att_tokens)
            out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " + con.strip() + " <START> " + sen.strip() + " <END>" + "\n"
            out_fp.write(out_str)
    

In [8]:
def process_file_v1(input_file, output_file):
    """
    The function process the data files for Delete & Generate and convert
    it for the Delete, Retrieve and Generate training by separating the content
    and attributes. 
    It randomly picks 70% of the attributes only to make the generation
    more realistic instead of just filling the blanks, which helps while generating
    sentences for test cases.
    
    Input_file: string : Path of the input file
    Output_file: string : Path of the output file 
    """
    with open(input_file) as fp:
        data = fp.read().splitlines()
    with open (output_file,"w") as out_fp:
        for x in tqdm(data):
            temp = x.split("<START>")
            con = temp[0].replace("<POS>","").replace("<NEG>","").replace("<CON_START>","")
            sen = temp[1].replace("<END>","")
            lt1 = con.split()
            lt2 = sen.split()
            att_words = [z for z in lt2 if z not in lt1]
            #print(att_words)
            
            if len(att_words) > 2:
                indx = np.array(list(range(len(att_words))))
                np.random.shuffle(indx)
                att_words = " ".join([att_words[indx[k]] for k in range(int(0.7 * len(att_words)))])
            else: # If attributes less than 2 then keep all the attributes
                att_words = " ".join(att_words)
            #print(att_words)
            # TODO HERE MODIFIY CON with <REPLACE> to show position of word
            # Also put Token <REPLACE> in reference file 
            
            out_str = "<ATTR_WORDS> " + att_words + " <CON_START> " + con.strip() + " <START> " + sen.strip() + " <END>" + "\n"
            out_fp.write(out_str)

In [6]:
'''Merge files'''
# Create sentiment_train file before run!
# Create sentiment_test file before run!
# Create sentiment_dev file before run!

data_dir = "/home/jack/Desktop/NN/clean/datasets/yelp"

filenames = [data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_1.txt", data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_0.txt"]
with open(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)


filenames = [data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt", data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt"]
with open(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

filenames = [data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_1.txt", data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_0.txt"]
with open(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev.txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [14]:

process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_1_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_0_all_attrs.txt")

100%|██████████| 177218/177218 [00:00<00:00, 316493.76it/s]


In [12]:
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_1_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0_all_attrs.txt")

100%|██████████| 500/500 [00:00<00:00, 269660.79it/s]


In [15]:
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_1_all_attrs.txt")
process_file(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_0_all_attrs.txt")

100%|██████████| 2000/2000 [00:00<00:00, 226621.14it/s]


In [16]:
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_1.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_train_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_train_0.txt")

100%|██████████| 177218/177218 [00:00<00:00, 216021.92it/s]


In [17]:
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_1.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_test_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_test_0.txt")

100%|██████████| 500/500 [00:00<00:00, 190269.64it/s]


In [18]:
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_0.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_1.txt")
process_file_v1(data_dir+"/processed_files_with_bert_with_best_head/sentiment_dev_1.txt",data_dir+"/processed_files_with_bert_with_best_head/delete_retrieve_edit_model/sentiment_dev_0.txt")

100%|██████████| 2000/2000 [00:00<00:00, 267553.60it/s]
